In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus
import graphviz

In [2]:
train_data = pd.read_csv("./decision_tree_pratice/titanic/train.csv")
test_data = pd.read_csv("./decision_tree_pratice/titanic/test.csv")
predict_data = pd.read_csv("./decision_tree_pratice/titanic/gender_submission.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data preprocessing

Drop some columns that might be useless.

In [3]:
train_data = train_data.drop('PassengerId', axis = 1)
train_data = train_data.drop('Name', axis = 1)
train_data = train_data.drop('Ticket', axis = 1)

test_data = test_data.drop('PassengerId', axis = 1)
test_data = test_data.drop('Name', axis = 1)
test_data = test_data.drop('Ticket', axis = 1)

## Change Sex string to number

In [4]:
sex_map = {'male' : 0, 'female' :1}
train_data['Sex'] = train_data['Sex'].map(sex_map)
test_data['Sex'] = test_data['Sex'].map(sex_map)

In [5]:
train_data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

## Deal with the NaN data
We first see the number of the NaN data in each column.

In [6]:
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

We could find that 'Cabin' has a lot of NaN data, so I decided to drop it off. <br>
We could all see that column 'Embarked' has two NaN data, so I would like to drop those two ID off.

In [7]:
train_data = train_data.drop('Cabin', axis = 1)
train_data.dropna(subset=['Embarked'], how='all', inplace = True)

Change Embarked string to float.

In [8]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [9]:
embarked_map = {'S' : 0, 'C' :1, 'Q': 2}
train_data['Embarked'] = train_data['Embarked'].map(embarked_map)
test_data['Embarked'] = test_data['Embarked'].map(embarked_map)

And see NaN number again, the number of 'Embarked' is zero.
Then, we can deal with the NaN of the column 'Age'.

In [10]:
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

We use the average of Age to fill in those NaN.

In [11]:
age = train_data['Age']
avgage = age.sum(axis = 0, skipna = True) /len(age)
train_data['Age'].fillna(avgage, inplace = True)

In [12]:
train_data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

## Seperate training data and testing data

In [13]:
train = train_data[:700]
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [14]:
test = train_data[700:]
test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
701,1,1,0,35.0,0,0,26.2875,0
702,0,3,1,18.0,0,1,14.4542,1
703,0,3,0,25.0,0,0,7.7417,2
704,0,3,0,26.0,1,0,7.8542,0
705,0,2,0,39.0,0,0,26.0000,0


seperate attributes and answer.

In [15]:
y = train['Survived'].values
train = train.drop('Survived', 1)
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,

# Decision Tree

In [19]:
dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(train),
                class_names=['die','survive'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

InvocationException: GraphViz's executables not found

In [18]:
dtree.feature_importances_

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
y_test = test['Survived'].values
X_test = test.drop('Survived', 1)

y_predict = dtree.predict(X_test)

y_predict

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)